## Summary

#### Цель:  
Выявить влияние показа товаров ТСП в карточках на заказы товаров Росско 

#### Задачи: 
 - Определить количество и долю просмотренных карточек товара с наличием товара ТСП, товара Росско и их сочетаний в самой карточке и кроссах  
 - Определить, количество и долю корзин с товарами ТСП и локальными товарами Росско  
 - Определить, из каких карточек были добавления в корзину товаров ТСП и товаров Росско (с наличием/отсутствием товара Росско в карточке и/или кроссе)   

#### Первичное необходимое действие: 
Организация ETL-процесса.  

#### О чем идет речь конкретно здесь:  
Ниже представлен процесс извлечения и первичной трансформации данных. Процесс загрузки о периодического повторения остается вне рассотрения (Место загрузки и формат конечной отчетности, ее обновления устанавливается отдельно). 

#### Детали:  
При анализе влияния ТСП на выручку рассматриваются 2 события:
1) просмотр карточки товара  
2) добавление товара в корзину  

У товаров, появившихся на карточке товара рассматриваются 2 признака:
1) Тип товара (искомый, кросс)  
2) Тип склада отгрузки (Росско, ТСП, и Росско и ТСП)  

У товаров, добавленных в корзину, рассматривается признак склада - ТСП это или локальный Росско. 
Также может рассматриваться признак покупки кросса или основного товара с карточки.
Товар, добавленный в корзину наследует признаки карточки товара, из которой был добавлен.

#### О возможном эксперименте:  
При запуске эксперимента, для определения влияния показа ТСП на продажу товаров Росско и совокупные продажи возможно отключить показ ТСП для тестовой группы и замерить разницу продаж товаров Росско и совокупных продаж с контрольной группой. Для обеспечения чистоты эксперимента желательно для тестовой группы помечать тип склада отгрузки, как реально показываемый, так и тот, который бы показывался в обычных условиях. При сплите в группах также желательно учитывать не только размер выручки, но и долю ТСП в просмотрах карточек и корзинах. Кроме того, необходимо учитывать кейсы, когда ТСП является единственным вариантом на карточке, а при его исключении карточка станет пустой  (останутся только кроссы с Росско или не будет даже их) - необходимо решение, показывать ли ТСП на таких карточках.

## Скрипт и комментарии

Далее представлен скрипт выгрузки данных о просмотрах карточек и добавлениях в корзину из Elasticsearch.  
Для примера взят Екатеринбург и данные за первые 2 дня 2024 года. При необходимости эти параметры настраиваются ручной сменой значений filial,start_date,control_date  
Доступ к базе Elasticsearch не указан в целях конфиденциальности.

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from pandas import json_normalize 
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Доступ к базе - используйте свой рабочий
es = Elasticsearch(
    ['какой-то_сервер'],
    http_auth=('какой-то_логин', 'какой-то_пароль'),
    timeout=100,
    max_retries=10,
    retry_on_timeout=True
)

In [3]:
#Эти параметры можно настроить под свои задачи
filial = ['Екатеринбург'] #Если филиалов несколько, то задаем их списком
start_date = "2024-01-01"
control_date = "2024-01-03"

##### Запросы на данные о событиях просмотров карточек товара с необходимыми параметрами

In [4]:
#Есть продукт и кросс
pcv_df_1=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "filial":{
                                            "terms":{
                                                "field":"region"
                                            }
                                        }
                                    },
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_product":{
                                            "terms":{
                                                "field":"delivery_types.product.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_crosses":{
                                            "terms":{
                                                "field":"delivery_types.crosses.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
pcv_df_1 = pd.concat([pcv_df_1,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "filial":{
                                                "terms":{
                                                    "field":"region"
                                                }
                                            }
                                        },
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_product":{
                                                "terms":{
                                                    "field":"delivery_types.product.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_crosses":{
                                                "terms":{
                                                    "field":"delivery_types.crosses.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    pcv_df_1=pd.concat([pcv_df_1,docs], join = 'outer', ignore_index=True)

In [5]:
#Нет ни продукта, ни кросса
pcv_df_2=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  },
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "filial":{
                                            "terms":{
                                                "field":"region"
                                            }
                                        }
                                    },
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
pcv_df_2 = pd.concat([pcv_df_2,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  },
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "filial":{
                                                "terms":{
                                                    "field":"region"
                                                }
                                            }
                                        },
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    pcv_df_2=pd.concat([pcv_df_2,docs], join = 'outer', ignore_index=True)

In [6]:
#Нет продукта, есть кросс
pcv_df_3=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  }
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "filial":{
                                            "terms":{
                                                "field":"region"
                                            }
                                        }
                                    },
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_crosses":{
                                            "terms":{
                                                "field":"delivery_types.crosses.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
pcv_df_3 = pd.concat([pcv_df_3,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  }
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "filial":{
                                                "terms":{
                                                    "field":"region"
                                                }
                                            }
                                        },
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_crosses":{
                                                "terms":{
                                                    "field":"delivery_types.crosses.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    pcv_df_3=pd.concat([pcv_df_3,docs], join = 'outer', ignore_index=True)

In [7]:
#Есть продукт, нет кросса
pcv_df_4=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  }
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "filial":{
                                            "terms":{
                                                "field":"region"
                                            }
                                        }
                                    },
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_product":{
                                            "terms":{
                                                "field":"delivery_types.product.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
pcv_df_4 = pd.concat([pcv_df_4,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['product_card_view']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "exists":{
                                          "field":"delivery_types.product"
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ],
                              "must_not":[
                                  {
                                      "exists":{
                                          "field":"delivery_types.crosses"
                                      }
                                  }
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "filial":{
                                                "terms":{
                                                    "field":"region"
                                                }
                                            }
                                        },
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_product":{
                                                "terms":{
                                                    "field":"delivery_types.product.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    pcv_df_4=pd.concat([pcv_df_4,docs], join = 'outer', ignore_index=True)

In [8]:
#Заполняем поля в неполных датасетах, устанавливаем единый порядок полей для конкатенации
pcv_df_2['key.delivery_types_product'] = 'Нет в наличии'
pcv_df_2['key.delivery_types_crosses'] = 'Нет в наличии'
pcv_df_3['key.delivery_types_product'] = 'Нет в наличии'
pcv_df_4['key.delivery_types_crosses'] = 'Нет в наличии'

pcv_df_1 = pcv_df_1[['key.filial','key.member_id','key.search_uid','key.nsi','key.delivery_types_product','key.delivery_types_crosses','key.date']]
pcv_df_2 = pcv_df_2[['key.filial','key.member_id','key.search_uid','key.nsi','key.delivery_types_product','key.delivery_types_crosses','key.date']]
pcv_df_3 = pcv_df_3[['key.filial','key.member_id','key.search_uid','key.nsi','key.delivery_types_product','key.delivery_types_crosses','key.date']]
pcv_df_4 = pcv_df_4[['key.filial','key.member_id','key.search_uid','key.nsi','key.delivery_types_product','key.delivery_types_crosses','key.date']]

In [9]:
pcv_df_final = pd.concat([pcv_df_1,
                          pcv_df_2,
                          pcv_df_3,
                          pcv_df_4],ignore_index=True)

##### Запрос на данные о добавлении товаров в корзину

In [10]:
#Выгружаем добавления в корзину с признаком источника nsi, типом склада
add_to_cart=pd.DataFrame()
result =es.search(index="cart_item_event-*", 
              body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['add_to_cart']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ]
                          }
              },
                    "aggs":{
                        "my_buckets":{
                            "composite":{
                                "sources":[
                                    {
                                        "member_id":{
                                            "terms":{
                                                "field":"member_id"
                                            }
                                        }
                                    },
                                    {
                                        "search_uid":{
                                            "terms":{
                                                "field":"search_uid.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "nsi":{
                                            "terms":{
                                                "field":"nsi.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_tsp":{
                                            "terms":{
                                                "field":"is_approximate_delivery_interval"
                                            }
                                        }
                                    },
                                    {
                                        "delivery_types_local":{
                                            "terms":{
                                                "field":"stock.is_local_delivery"
                                            }
                                        }
                                    },
                                    {
                                        "product":{
                                            "terms":{
                                                "field":"product.path.keyword"
                                            }
                                        }
                                    },
                                    {
                                        "date":{
                                            "date_histogram": {
                                                "field": "server_timestamp",
                                                "calendar_interval": "1d",
                                                "format": "yyyy-MM-dd"
                                            }
                                        }
                                    }
                                ],
                                "size" : 1000
                            }
                        }
                    }
                   }
             )
g = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
add_to_cart = pd.concat([add_to_cart,g], join = 'outer', ignore_index=True)
while 'after_key' in list(result['aggregations']['my_buckets'].keys()):
    k = result['aggregations']['my_buckets']['after_key']
    result = es.search(index="cart_item_event-*", 
                  body={"query":{
                          "bool":{
                              "must": [
                                  {
                                      "terms":{
                                          "event": ['add_to_cart']
                                      }
                                  },
                                  {
                                      "terms":{
                                          "region": filial
                                      }
                                  },
                                  {
                                      "match":{
                                           "is_internal_user": False
                                      }
                                  },
                                  {
                                      "range":{
                                          "server_timestamp":{
                                              "gte": start_date,"lt": control_date,
                                              "format": "yyyy-MM-dd"
                                          }
                                      }
                                  }
                              ]
                          }
              },
                        "aggs":{
                            "my_buckets":{
                                "composite":{
                                    "sources":[
                                        {
                                            "member_id":{
                                                "terms":{
                                                    "field":"member_id"
                                                }
                                            }
                                        },
                                        {
                                            "search_uid":{
                                                "terms":{
                                                    "field":"search_uid.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "nsi":{
                                                "terms":{
                                                    "field":"nsi.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_tsp":{
                                                "terms":{
                                                    "field":"is_approximate_delivery_interval"
                                                }
                                            }
                                        },
                                        {
                                            "delivery_types_local":{
                                                "terms":{
                                                    "field":"stock.is_local_delivery"
                                                }
                                            }
                                        },
                                        {
                                            "product":{
                                                "terms":{
                                                    "field":"product.path.keyword"
                                                }
                                            }
                                        },
                                        {
                                            "date":{
                                                "date_histogram": {
                                                    "field": "server_timestamp",
                                                    "calendar_interval": "1d",
                                                    "format": "yyyy-MM-dd"
                                                }
                                            }
                                        }
                                    ],
                                    "size" : 1000,
                                    "after": k
                                }
                            }
                        }
                       }
                 )
    docs = pd.json_normalize(result['aggregations']['my_buckets']['buckets'])
    add_to_cart=pd.concat([add_to_cart,docs], join = 'outer', ignore_index=True)

In [11]:
#Выделяем НСИ на карточке, из которой было добавление в корзину
add_to_cart['key.product']=add_to_cart['key.product'].astype(str).map(lambda x: x.lstrip('/product?text='))

In [12]:
#Разграничение НСИ карточки и корзины, подготовка мерджа карточки и корзины
add_to_cart.rename(columns={'key.nsi':'key.nsi_cart'},inplace=True)
add_to_cart.rename(columns={'key.product':'key.nsi'},inplace=True)
add_to_cart['is_cart']=1

##### Матчинг карточек товаров и корзины

In [13]:
new_df = pcv_df_final.merge(add_to_cart[['key.member_id',
                                         'key.search_uid',
                                         'key.nsi',
                                         'key.nsi_cart',
                                         'key.delivery_types_tsp',
                                         'key.delivery_types_local',
                                         'is_cart']], how='left', on = ['key.member_id',
                                                                     'key.search_uid',
                                                                     'key.nsi'])


In [14]:
new_df['is_cart'].fillna(0,inplace=True)
new_df['key.delivery_types_tsp'].fillna(False,inplace=True)
new_df['key.delivery_types_local'].fillna(False,inplace=True)
new_df['key.delivery_types_local'].fillna(False,inplace=True)
new_df['key.nsi_cart'].fillna('empty',inplace=True)

In [15]:
new_df['is_cross'] = False
new_df.loc[(new_df['key.nsi_cart']!='empty')&(new_df['key.nsi_cart']==new_df['key.nsi']),'is_cross']=True

##### В результате преобразований получаем следующий набор данных, пригодный к загрузке в хранилище для использования в построении отчетов

In [16]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2515 entries, 0 to 2514
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   key.filial                  2515 non-null   object 
 1   key.member_id               2515 non-null   int64  
 2   key.search_uid              2515 non-null   object 
 3   key.nsi                     2515 non-null   object 
 4   key.delivery_types_product  2515 non-null   object 
 5   key.delivery_types_crosses  2515 non-null   object 
 6   key.date                    2515 non-null   object 
 7   key.nsi_cart                2515 non-null   object 
 8   key.delivery_types_tsp      2515 non-null   bool   
 9   key.delivery_types_local    2515 non-null   bool   
 10  is_cart                     2515 non-null   float64
 11  is_cross                    2515 non-null   bool   
dtypes: bool(3), float64(1), int64(1), object(7)
memory usage: 203.9+ KB


In [17]:
new_df.head()

,key.filial,key.member_id,key.search_uid,key.nsi,key.delivery_types_product,key.delivery_types_crosses,key.date,key.nsi_cart,key.delivery_types_tsp,key.delivery_types_local,is_cart,is_cross
0,Екатеринбург,8160,020e3f6c-44a8-46bc-b718-b1b1c0ad5d09,NSIN0023169008,ТСП/Росско,ТСП/Росско,2024-01-02,NSIN0023169008,False,True,1.0,True
1,Екатеринбург,8160,17f229cb-b28c-4f89-8415-6d31623e9256,NSIN0023169008,ТСП/Росско,ТСП/Росско,2024-01-02,NSIN0023169008,False,True,1.0,True
2,Екатеринбург,8160,180ac1b2-ab3d-4b4d-be2b-384129da10d5,NSII0007955452,ТСП/Росско,ТСП/Росско,2024-01-02,NSII0007955452,False,True,1.0,True
3,Екатеринбург,8160,2d64dc4e-3ebe-4ea1-a251-ced326eb45af,NSII0007955452,ТСП/Росско,ТСП/Росско,2024-01-02,empty,False,False,0.0,False
4,Екатеринбург,8160,3f98f53b-aeb2-4626-8b6a-248b8178471a,NSII0023283917,Росско,ТСП/Росско,2024-01-02,empty,False,False,0.0,False


##### Для примера на этом фрагменте данных можем посмотреть, сколько и каких (с ТСП/Росско в карточке/кроссах) было открыто карточек товара, сколько и каких товаров было добавлено в корзину, из каких карточек и от куда (из кроссов или искомого товара)  
Набор данных позволяет также при необходимости характеризовать пользователей по параметрам просматриваемых ими карточек и добавлениям в корзину

In [18]:
new_df[['key.delivery_types_product','key.delivery_types_crosses','key.delivery_types_tsp','key.delivery_types_local','is_cart']].value_counts()

key.delivery_types_product  key.delivery_types_crosses  key.delivery_types_tsp  key.delivery_types_local  is_cart
ТСП                         ТСП/Росско                  False                   False                     0.0        386
Росско                      ТСП/Росско                  False                   False                     0.0        288
Нет в наличии               ТСП/Росско                  False                   False                     0.0        244
                            Нет в наличии               False                   False                     0.0        222
ТСП/Росско                  ТСП/Росско                  False                   True                      1.0        218
Росско                      ТСП/Росско                  False                   True                      1.0        194
ТСП/Росско                  ТСП/Росско                  False                   False                     0.0        179
ТСП                         Нет в налич

In [19]:
new_df.loc[new_df['is_cart']==1,['key.delivery_types_product','key.delivery_types_crosses','key.delivery_types_tsp','key.delivery_types_local']].value_counts()

key.delivery_types_product  key.delivery_types_crosses  key.delivery_types_tsp  key.delivery_types_local
ТСП/Росско                  ТСП/Росско                  False                   True                        218
Росско                      ТСП/Росско                  False                   True                        194
ТСП                         ТСП/Росско                  False                   True                         89
Нет в наличии               ТСП/Росско                  False                   True                         48
ТСП/Росско                  ТСП/Росско                  False                   False                        20
ТСП                         ТСП/Росско                  True                    False                        19
                            Нет в наличии               True                    False                        17
                            ТСП                         True                    False                        15